<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/TokenClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import nltk
from sklearn.metrics import classification_report
from nltk.tokenize import TreebankWordTokenizer as twt
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [ ]:
!nvidia-smi

Fri Jun 24 16:04:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.11.0+cu113
GPU: Tesla T4


# Load Data

In [ ]:
iob_train = pd.read_csv('/content/iob_train.csv')
iob_test = pd.read_csv('/content/iob_test.csv')
iob_dev = pd.read_csv('/content/iob_dev.csv')

# Finetune Models

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 5.73 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 132 kB 5.0 MB/s 
     |████████████████████████████████| 8.9 MB 41.6 MB/s 
     |████████████████████████████████| 79 kB 11.2 MB/s 
     |████████████████████████████████| 138 kB 64.8 MB/s 


In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
# combine train and dev
train = iob_train.append(iob_dev, ignore_index=True)
train = train.reset_index()
print("Train and dev data: %d sentences, %d tokens"%(len(train), len(flatten(train.tokens))))
print("Test data: %d sentences, %d tokens"%(len(iob_test), len(flatten(iob_test.tokens))))

Train and dev data: 981 sentences, 203599 tokens
Test data: 497 sentences, 107867 tokens


In [ ]:
X_train, y_train = train.tokens, train.iob_tags
X_test, y_test = iob_test.tokens, iob_test.iob_tags

In [ ]:
label_list = list(np.unique(flatten(y_train)))
print("\nNER tags:",label_list)


NER tags: ['B-Chemical', 'B-Disease', 'I-Chemical', 'I-Disease', 'O']


In [ ]:
train.head()

,index,tokens,iob_tags
0,0,"[Naloxone, reverses, the, antihypertensive, ef...","[B-Chemical, O, O, O, O, O, B-Chemical, O, O, ..."
1,1,"[Lidocaine-induced, cardiac, asystole., Intrav...","[B-Chemical, B-Disease, O, O, O, O, O, O, O, O..."
2,2,"[Suxamethonium, infusion, rate, and, observed,...","[B-Chemical, O, O, O, O, B-Disease, O, O, O, B..."
3,3,"[Galanthamine, hydrobromide, ,, a, longer, act...","[B-Chemical, I-Chemical, O, O, O, O, O, O, O, ..."
4,4,"[Effects, of, uninephrectomy, and, high, prote...","[O, O, O, O, O, O, O, O, B-Chemical, B-Disease..."


In [ ]:
example_tokens = X_test[0]
example_labels = y_test[0]

data = {"token": example_tokens,"label": example_labels}
df=pd.DataFrame(data=data)
df.head(50)

,token,label
0,Famotidine-associated,B-Chemical
1,delirium.,B-Disease
2,A,O
3,series,O
4,of,O
5,six,O
6,cases.,O
7,Famotidine,B-Chemical
8,is,O
9,a,O


### BERT

1) Use `'bert-base-cased'` model 

2) Set the **`ignore_label`** =`['O']`. Exclude the non named entities.

3) Set the **` max_seq_len`** = 256 - high enough to cover lengths of the token lists

In [ ]:
%%time
bert_model = BertTokenClassifier('bert-base-cased',
                            max_seq_length=256,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0.,                            
                            label_list=label_list,
                            ignore_label=['O'])
bert_model.fit(X_train, y_train)

Building sklearn token classifier...
Loading bert-base-cased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 981, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

CPU times: user 1min 43s, sys: 43 s, total: 2min 26s
Wall time: 2min 55s


In [ ]:
bert_f1_test = bert_model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(bert_f1_test))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

Test f1: 70.32


In [ ]:
bert_y_preds = bert_model.predict(X_test)
print(classification_report(flatten(y_test), flatten(bert_y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  B-Chemical       0.92      0.66      0.77      5240
   B-Disease       0.82      0.61      0.70      4337
  I-Chemical       0.74      0.63      0.68       530
   I-Disease       0.70      0.63      0.67      2142
           O       0.96      0.99      0.97     95618

    accuracy                           0.95    107867
   macro avg       0.83      0.70      0.76    107867
weighted avg       0.94      0.95      0.94    107867



In [ ]:
iter_zip = zip(flatten(X_test),flatten(y_test),flatten(bert_y_preds))
preds = [" ".join([token, y, y_pred]) for token, y, y_pred in iter_zip]
with open("preds.txt",'w') as f:
    for x in preds:
        f.write(str(x)+'\n') 
 
!perl ./other_examples/conlleval.pl < preds.txt
!rm preds.txt

processed 107867 tokens with 9582 phrases; found: 7184 phrases; correct: 5864.
accuracy:  94.76%; precision:  81.63%; recall:  61.20%; FB1:  69.95
         Chemical: precision:  89.17%; recall:  64.39%; FB1:  74.78  3786
          Disease: precision:  73.22%; recall:  57.34%; FB1:  64.31  3398


### SciBERT

1) Use `'scibert-scivocab-cased'` model. [GitHub](https://github.com/allenai/scibert), [paper](https://arxiv.org/pdf/1903.10676.pdf)

2) Set the **`ignore_label`** =`['O']`. Exclude the non named entities.

3) Set the **` max_seq_len`** = 256 - high enough to cover lengths of the token lists

In [ ]:
%%time
scibert_model = BertTokenClassifier('scibert-scivocab-cased',
                            max_seq_length=256,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0.,                            
                            label_list=label_list,
                            ignore_label=['O'])



scibert_model.fit(X_train, y_train)

Building sklearn token classifier...
Loading scibert-scivocab-cased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 981, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

CPU times: user 1min 54s, sys: 49.7 s, total: 2min 44s
Wall time: 3min 14s


In [ ]:
scibert_f1_test = scibert_model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(scibert_f1_test))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

Test f1: 76.97


In [ ]:
scibert_y_preds = scibert_model.predict(X_test)
print(classification_report(flatten(y_test), flatten(scibert_y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  B-Chemical       0.95      0.75      0.84      5240
   B-Disease       0.86      0.71      0.78      4337
  I-Chemical       0.78      0.69      0.73       530
   I-Disease       0.75      0.70      0.73      2142
           O       0.97      0.99      0.98     95618

    accuracy                           0.96    107867
   macro avg       0.86      0.77      0.81    107867
weighted avg       0.96      0.96      0.96    107867



In [ ]:
iter_zip = zip(flatten(X_test),flatten(y_test),flatten(scibert_y_preds))
preds = [" ".join([token, y, y_pred]) for token, y, y_pred in iter_zip]
with open("preds.txt",'w') as f:
    for x in preds:
        f.write(str(x)+'\n') 

!perl ./other_examples/conlleval.pl < preds.txt
!rm preds.txt

processed 107867 tokens with 9582 phrases; found: 7904 phrases; correct: 6845.
accuracy:  95.95%; precision:  86.60%; recall:  71.44%; FB1:  78.29
         Chemical: precision:  93.02%; recall:  74.19%; FB1:  82.55  4182
          Disease: precision:  79.39%; recall:  68.10%; FB1:  73.32  3722


### BioBERT
1) Use `biobert-v1.1-pubmed-base-cased` model. [BioBERT github](https://github.com/dmis-lab/biobert), [paper](https://arxiv.org/pdf/1901.08746.pdf)

2) Set the **`ignore_label`** =`['O']`. Exclude the non named entities.

3) Set the **` max_seq_len`** = 256 - high enough to cover lengths of the token lists

In [ ]:
%%time
biobert_model = BertTokenClassifier('biobert-v1.1-pubmed-base-cased',
                            max_seq_length=256,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0,                            
                            label_list=label_list,
                            ignore_label=['O'])

biobert_model.fit(X_train, y_train)

Building sklearn token classifier...
Loading biobert-v1.1-pubmed-base-cased model...
Defaulting to linear classifier/regressor
Loading Tensorflow checkpoint from  model.ckpt-1000000

train data size: 981, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

CPU times: user 1min 50s, sys: 45 s, total: 2min 35s
Wall time: 3min 4s


In [ ]:
biobert_f1_test = biobert_model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(biobert_f1_test))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

Test f1: 73.07


In [ ]:
biobert_y_preds = biobert_model.predict(X_test)
print(classification_report(flatten(y_test), flatten(biobert_y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  B-Chemical       0.95      0.67      0.79      5240
   B-Disease       0.86      0.65      0.74      4337
  I-Chemical       0.80      0.64      0.71       530
   I-Disease       0.72      0.66      0.69      2142
           O       0.96      0.99      0.97     95618

    accuracy                           0.95    107867
   macro avg       0.86      0.72      0.78    107867
weighted avg       0.95      0.95      0.95    107867



In [ ]:
iter_zip = zip(flatten(X_test),flatten(y_test),flatten(biobert_y_preds))
preds = [" ".join([token, y, y_pred]) for token, y, y_pred in iter_zip]
with open("preds.txt",'w') as f:
    for x in preds:
        f.write(str(x)+'\n') 


!perl ./other_examples/conlleval.pl < preds.txt
!rm preds.txt

processed 107867 tokens with 9582 phrases; found: 7048 phrases; correct: 6024.
accuracy:  95.20%; precision:  85.47%; recall:  62.87%; FB1:  72.45
         Chemical: precision:  93.34%; recall:  65.76%; FB1:  77.16  3694
          Disease: precision:  76.80%; recall:  59.37%; FB1:  66.97  3354


# Compare Models

In [ ]:
num = 12
tokens = X_test[num]
labels = y_test[num]

bert_preds  = bert_y_preds[num]
scibert_preds  = scibert_y_preds[num]
biobert_preds  = biobert_y_preds[num]

In [ ]:
data = {"token": tokens,"label": labels, "bert": bert_preds,"scibert": scibert_preds, "biobert": biobert_preds}
df=pd.DataFrame(data=data)
df.head(50)

,token,label,bert,scibert,biobert
0,Risk,O,O,O,O
1,factors,O,O,O,O
2,and,O,O,O,O
3,predictors,O,O,O,O
4,of,O,O,O,O
5,levodopa-induced,B-Chemical,B-Chemical,B-Chemical,B-Chemical
6,dyskinesia,B-Disease,B-Disease,B-Disease,B-Disease
7,among,O,O,O,O
8,multiethnic,O,O,O,O
9,Malaysians,O,O,O,O
